In [202]:
# imported mysql.connector to be able to connect the database
import mysql.connector as mysql
from functios_for_api import upload_berryfile_to_mysql

# data management imports
import numpy as np
import pandas as pd
import seaborn as sns

ImportError: cannot import name 'upload_berryfile_to_mysql' from 'functios_for_api' (c:\Users\Niko\OneDrive - lucit\First year\Spring-semester\project\project work\Semester-Project\python\functios_for_api.py)

<h3>getting the data cleaned.</h3>

In [196]:
# this cleanup is the sameone Tuomas made in the classes.

In [197]:
# load data in DataFrame
df = pd.read_excel("LUKE_ripe_berries_observation_dates.xlsx")

# translate some columns
df = df.rename(columns={"marja": "berry", "havainto pvm": "observation_date", "pääpuulaji": "main_tree_type",
         "muu puulaji": "secondary_tree_type", "metsän kehitysluokka": "forest_development_class"})

# drop unneeded columns (for now)
df = df.drop("vaihe", axis=1)
df = df.drop("Välimatka karkeistetun ja todellisen sijainnin välillä", axis=1)

# translate the berry names
df['berry'] = df['berry'].replace(['mustikka'], 'bilberry')
df['berry'] = df['berry'].replace(['puolukka'], 'lingonberry')
df['berry'] = df['berry'].replace(['suomuurain'], 'cloudberry')

# translate main tree type
df['main_tree_type'] = df['main_tree_type'].replace(['koivu'], 'birch')
df['main_tree_type'] = df['main_tree_type'].replace(['kuusi'], 'spruce')
df['main_tree_type'] = df['main_tree_type'].replace(['mänty'], 'pine')
df['main_tree_type'] = df['main_tree_type'].replace(['muu'], 'other')

# helper function to replace "other" with the secondary tree type
def replace_other_tree_type(row):
    if row['main_tree_type'] == "other":
        return row['secondary_tree_type'].lower().replace(" ", "")
    else:
        return row['main_tree_type']


df['main_tree_type'] = df.apply(replace_other_tree_type, axis=1)

# translating the secondary forest types too
df['main_tree_type'] = df['main_tree_type'].replace('suo', 'swamp')
df['main_tree_type'] = df['main_tree_type'].replace('suota', 'swamp')
df['main_tree_type'] = df['main_tree_type'].replace('suopohja', 'swamp')
df['main_tree_type'] = df['main_tree_type'].replace('sekametsä', 'mixed')
df['main_tree_type'] = df['main_tree_type'].replace('seka', 'mixed')
df['main_tree_type'] = df['main_tree_type'].replace('sekapuusto', 'mixed')
df['main_tree_type'] = df['main_tree_type'].replace('lehtikuusi', 'larch')
df['main_tree_type'] = df['main_tree_type'].replace('aukea', 'open')
df['main_tree_type'] = df['main_tree_type'].replace('aukio', 'open')
df['main_tree_type'] = df['main_tree_type'].replace('eipuita', 'open')
df['main_tree_type'] = df['main_tree_type'].replace('puuton', 'open')
df['main_tree_type'] = df['main_tree_type'].replace('vanhaavaivaiskoivua', 'softbirch')
df['main_tree_type'] = df['main_tree_type'].replace('vaivaiskoivikko', 'softbirch')

# now we only have values in English
df['main_tree_type'].value_counts()

df['main_tree_type'] = df['main_tree_type'].astype(str)

df = df.drop("secondary_tree_type", axis=1)

# kasvatusmetsä = plantation_forest
# taimikko = seedling
# varttunut metsä = old_forest
# aukea = clearing

# translate the forest development class
df['forest_development_class'] = df['forest_development_class'].replace(['aukea'], 'clearing')
df['forest_development_class'] = df['forest_development_class'].replace(['taimikko'], 'seedling')
df['forest_development_class'] = df['forest_development_class'].replace(['varttunut metsä'], 'old')
df['forest_development_class'] = df['forest_development_class'].replace(['kasvatusmetsä'], 'plantation')

# split the observation date into year, month, day and also a week number
df['year'] = df['observation_date'].dt.year
df['month'] = df['observation_date'].dt.month
df['day'] = df['observation_date'].dt.day
df['week_number'] = df['observation_date'].dt.isocalendar().week

df['berry'] = df['berry'].astype(str)
df['berry'].dtype


dtype('O')

In [198]:
df

,berry,observation_date,main_tree_type,forest_development_class,x,y,year,month,day,week_number
0,cloudberry,2018-05-27,pine,old,400000,7320000,2018,5,27,21
1,cloudberry,2018-06-28,birch,clearing,470000,7150000,2018,6,28,26
2,bilberry,2018-07-04,mixed,plantation,450000,7310000,2018,7,4,27
3,bilberry,2018-07-06,pine,old,430000,6910000,2018,7,6,27
4,bilberry,2018-07-07,pine,old,410000,7040000,2018,7,7,27
...,...,...,...,...,...,...,...,...,...,...
882,lingonberry,2022-09-17,pine,plantation,390000,6730000,2022,9,17,37
883,lingonberry,2022-09-18,pine,old,420000,7140000,2022,9,18,37
884,lingonberry,2022-09-20,spruce,plantation,300000,6900000,2022,9,20,38
885,lingonberry,2022-09-21,pine,old,570000,7720000,2022,9,21,38


<h3>data to the backend</h3>
<h3></h3>

In [199]:
# connecting to local host
mydb = mysql.connect(
        host="localhost",
        port="3309",
        user="root",
        password="",
        db="mysql"
        )

ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)

In [ ]:
# select the correct database here
mycursor = mydb.cursor()

In [ ]:
# creates the database
mycursor.execute("CREATE DATABASE Berry_nice")

In [ ]:
# creating the table
mycursor.execute("CREATE TABLE berryes (log_id INT, berry VARCHAR(255), state VARCHAR(255), observation_date Datetime, main_tree_typee VARCHAR(255), forest_development_class VARCHAR(255),x_cord INT, y_cord INT, year INT, month INT, day INT, week_number INT, PRIMARY KEY (log_id))")


In [ ]:
def upload_berryfile_to_mysql(data):
    # mycursor.execute("CREATE TABLE berryes (log_id INT, specie VARCHAR(255), state VARCHAR(255), date_of_observation Datetime, main_forest_type VARCHAR(255), other_forest_type VARCHAR(255), forest_development_class VARCHAR(255),x_cord INT, y_cord INT, PRIMARY KEY (log_id))")

    # return(data)  
    id = 0
    for index, row in data.iterrows():
        id += 1

        val = [(f'{id}, {row["berry"]}, {row["state"]}, {row["observation_date"]}, {row["main_tree_typee"]}, {row["forest_development_class"]}, {row["x_cord"]}, {row["y_cord"]}, {row["year"]}, {row["month"]}, {row["day"]}, {row["week_number"]}')]

        print(val)


In [200]:
upload_berryfile_to_mysql(df)

AttributeError: module 'functios_for_api' has no attribute 'upload_berryfile_to_mysql'